# Bulk Labelling as a Notebook

This notebook contains a convenient pattern to cluster and label new text data. The end-goal is to discover intents that might be used in a virtual assistant setting. This can be especially useful in an early stage and is part of the "iterate on your data"-mindset. 

## Dependencies 

You'll need to install a few things to get started. 

- [whatlies](https://rasahq.github.io/whatlies/)
- [human-learn](https://koaning.github.io/human-learn/)

You can install both tools by running this line in an empty cell; 

```python
%pip install "whatlies[tfhub]" "human-learn"
```

We use `whatlies` to fetch embeddings and to handle the dimensionality reduction. We use `human-learn` for the interactive labelling interface. Feel free to check the documentation of both packages to learn more. 

## Let's go

To get started we'll first import a few tools.

In [15]:
import pathlib 
import numpy as np
import tensorflow_hub
from whatlies.language import CountVectorLanguage, UniversalSentenceLanguage, BytePairLanguage, SentenceTFMLanguage
from whatlies.language import TFHubLanguage
from whatlies import Embedding
from whatlies.transformers import Pca, Umap
import json

In [25]:
import datasets
emotion_data=datasets.load_dataset('emotion')
app_data=datasets.load_dataset('app_reviews')
coronavirus_queries=datasets.load_dataset("bing_coronavirus_query_set", queries_by="country", start_date="2020-09-01", end_date="2020-09-30")



Using custom data configuration default
Reusing dataset emotion (/Users/julienguinot/.cache/huggingface/datasets/emotion/default/0.0.0/aa34462255cd487d04be8387a2d572588f6ceee23f784f37365aa714afeb8fe6)
Using custom data configuration default
Reusing dataset app_reviews (/Users/julienguinot/.cache/huggingface/datasets/app_reviews/default/0.0.0/af305ac963fd8dff5976dd341e97edf3a2933c3509a58885caace361c5cd3fe3)
Using custom data configuration country_2020-09-01_2020-09-30-82e6da681e69e488
Reusing dataset bing_coronavirus_query_set (/Users/julienguinot/.cache/huggingface/datasets/bing_coronavirus_query_set/country_2020-09-01_2020-09-30-82e6da681e69e488/0.0.0/8a91cd8df47019f989562a6af8e7086010014b6e0a01e2dd5643e7b40eb23a53)


In [26]:
import pandas as pd
emotion_data=pd.DataFrame.from_dict(emotion_data['train'])
app_data=pd.DataFrame.from_dict(app_data['train'])
coronavirus_queries=pd.DataFrame.from_dict(coronavirus_queries['train'])

In [32]:
coronavirus_queries.head()

,Country,Date,IsImplicitIntent,PopularityScore,Query,id
0,Romania,2020-09-01,False,3,coronavirus worldometer,1
1,United States,2020-09-01,True,1,scdhec,2
2,United States,2020-09-01,False,1,n95 mask coronavirus,3
3,Brazil,2020-09-01,True,28,parcelamento fgts mp 927,4
4,United States,2020-09-01,False,1,coronavirus colombia,5


Next we will load in some embedding frameworks. There can be very heavy, just so you know! 

In [19]:
lang_cv  = CountVectorLanguage(10)
lang_use = TFHubLanguage('universal-sentence-encoder_4')
lang_bp  = BytePairLanguage("en", dim=300, vs=200_000)

Next we'll load in the texts that we'd like to embed/cluster. The goal here is to provide multiple datasets to test varying functionality of the algorithm in various cases.

In [45]:
#Original dataset : very clear and idealistic
#txt = pathlib.Path("nlu.md").read_text()
#texts = list(set([t.replace(" - ", "") for t in txt.split("\n") if len(t) > 0 and t[0] != "#"]))
#print(f"We're going to plot {len(texts)} texts.")

#emotions dataset : tweets showing a range of five emotions
texts=emotion_data.text.tolist()[0:3000]

#app review dataset
texts=app_data.review.tolist()[0:3000]

#corovirus queries dataset
texts=coronavirus_queries.head(3000)['Query'].tolist()

Keep in mind that it's better to start out with 1000 sentences or so. Much more might break the browser's memory in the next visual.

## Showing Clusters 

![](pipeline.png)

The cell below will take the texts and have them pass through different language backends. After this they will be mapped to a two dimensional space by using [UMAP](https://umap-learn.readthedocs.io/en/latest/). It takes a while to plot everything (mainly because the universal sentence encoder and the transformer language models are heavy).

In [46]:
%%time
import warnings

def make_plot(lang):
    return (lang[texts]
             .transform(Umap(2))
             .plot_interactive(annot=False)
             .properties(width=200, height=200, title=type(lang).__name__))
make_plot(lang_use) | make_plot(lang_cv) | make_plot(lang_bp)  

CPU times: user 1min 9s, sys: 1.74 s, total: 1min 11s
Wall time: 43.3 s


alt.HConcatChart(...)

What you see are four charts. You should notice that certain clusters have appeared. For your usecase you might need to check which language backend makes the most sense. 

## Note for Non-English 

The only model shown here that is English specific is the universal sentence encoder (`lang_use`). All the other ones also support other languages. For more information check the [bytepair documentation](https://nlp.h-its.org/bpemb/) and the [sentence transformer documentation](https://www.sbert.net/docs/pretrained_models.html#multi-lingual-models).

## Towards Labelling 

We'll now prepare a dataframe that we'll assign labels to. We'll do that by loading in the same text file but now into a pandas dataframe.

In [6]:
df = lang_use[texts].transform(Umap(2)).to_dataframe().reset_index()
df.columns = ['text', 'd1', 'd2']
df['label'] = ''
df.shape[0]

1087

We are now going to be labelling!

# Fancy interactive drawing! 

We'll be using Vincent's infamous [human-learn library](https://koaning.github.io/human-learn/guide/drawing-features/custom-features.html) for this. First we'll need to instantiate some charts.

Next we get to draw! Drawing can be a bit tricky though, so pay attention. 

1. You'll want to double-click to start drawing. 
2. You can then click points together to form a polygon. 
3. Next you need to double-click to stop drawing. 

This allows you to draw polygons that can be used in the code below to fetch the examples that you're interested in.

## Rerun

This is where we will start labelling. That also means that we might re-run this cell after we've added labels.

In [26]:
from hulearn.experimental.interactive import InteractiveCharts

charts = InteractiveCharts(df.loc[lambda d: d['label'] == ''], labels=['group'])

charts.add_chart(x='d1', y='d2')

Loading BokehJS ...

We can now use this selection to retreive a subset of rows. This is a quick varification to see if the points you select indeed belong to the same cluster.

In [23]:
from hulearn.preprocessing import InteractivePreprocessor
tfm = InteractivePreprocessor(json_desc=charts.data())

df.pipe(tfm.pandas_pipe).loc[lambda d: d['group'] != 0].sample(10)

,text,d1,d2,label,group
195,can you tell me your age?,8.330536,-7.792078,,1
190,From where did you come?,8.228561,-4.920124,,1
947,Where did you come from?,8.244767,-4.900227,,1
151,what is your birthday?,7.743145,-7.129227,,1
479,how old are u,7.773867,-7.961021,,1
797,What area are you from?,8.292660,-5.523428,,1
461,can you tell me what number represents your age?,8.243992,-7.800375,,1
663,What city are you in?,8.151293,-5.675588,,1
120,do you know how old you are?,7.778389,-7.997067,,1
141,what is your exact age?,8.266831,-7.720518,,1


If you're confident that you'd like to assign a label, you can do so below. 

In [24]:
label_name = 'origin'

In [25]:
idx = df.pipe(tfm.pandas_pipe).loc[lambda d: d['group'] != 0].index

df.iloc[idx, 3] = label_name

print(f"We just assigned {len(idx)} labels!")

We just assigned 100 labels!


That's it! You've just attached a label to a group of points! 

## Rerun 

You can now scroll up and start relabelling clusters that aren't assigned yet. Once you're confident that this works, you can export by running the final code below.

In [22]:
df.head()

,text,d1,d2,label
0,What languages can you communicate in?,2.727605,-0.036231,
1,what u can do?,10.641682,5.367977,
2,What exactly is my name?,14.029011,1.462056,
3,What does Rasa make?,24.505444,6.782425,
4,can you help me?,8.584467,4.582135,


In [168]:
df.to_csv("first_order_labelled.csv")

## Final Notes

There's a few things to mention. 

1. This method of labelling is great when you're working on version 0 of something. It'll get you a whole lot of data fast but it won't be high quality data. 
2. The use-case for this method might be at the start of design a virtual assistant. You've probably got data from social media that you'd like to use as a source of inspiration for intents. This is certainly a valid starting point but you should be aware that the language that folks use on a feedback form is different than the language used in a chatbox. Again, these labels are a reasonable starting point, but they should not be regarded as ground truth. 
3. Labelling is only part of the goal here. Another big part is understanding the data. This is very much a qualitative/human task. You might be able to quickly label 1000 points in 5 minutes with this technique but you'll lack an understanding if you don't take the time for it. 